In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from stockMarket.core import get_tickers_from_index
from stockMarket.api import populate_contracts

tickers = get_tickers_from_index('sp500')
# contracts = populate_contracts(tickers)

In [62]:
from stockMarket.sec_edgar.downloader import Downloader

tickers = [ticker.replace(".", "-") for ticker in tickers]

dl = Downloader(tickers=tickers)
dl.download_10k_reports(fast_download=True)
dl.download_financial_statements_xlsx(from_year=2014)

 50%|████▉     | 249/503 [00:00<00:00, 2488.53it/s]/home/jag/projects/stockMarket/stockMarket/sec_edgar/downloader.py:51: UserWarning: No 10-K reports found for KVUE with cik 0001944048
  warnings.warn(message, UserWarning)
100%|██████████| 503/503 [00:00<00:00, 2730.21it/s]


In [125]:
import glob

from stockMarket.sec_edgar.financialReportReader import FinancialReportReader
from tqdm import tqdm

for ticker in tqdm(tickers):
    files = glob.glob(f"../data/sec-edgar-filings/{ticker}/10-K/*/Financial_Report.xlsx")
    
    if ticker in ["MTB", "NVR"]:
        continue
    
    #something wrong with the statement
    if ticker in ["AEE"]:
        continue

    for file in files:
        if 14 < int(file.split("-")[-2]) <= 24:
            reader = FinancialReportReader(file)
            reader.read()

  3%|▎         | 16/503 [00:59<30:04,  3.71s/it]


ValueError: No revenue found for file ../data/sec-edgar-filings/AEE/10-K/0001002910-21-000065/Financial_Report.xlsx